In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X为样本特征，y为样本类别输出， 共10000个样本，每个样本20个特征，输出有2个类别，没有冗余特征，每个类别一个簇

In [3]:
from sklearn.datasets.samples_generator import make_classification

X, y = make_classification(n_samples=10000, n_features=20, n_redundant=0,
                             n_clusters_per_class=1, n_classes=2, flip_y=0.1)

In [5]:
X

array([[-0.10105434, -0.44625145, -0.16018311, ...,  0.64440604,
         0.11340824, -0.46350682],
       [-0.9876784 ,  1.39616527, -1.82928894, ..., -0.1752953 ,
        -0.99810912,  0.99595856],
       [-0.00458682,  0.91131355, -0.23300418, ...,  0.63599969,
        -0.09860683, -0.39481746],
       ...,
       [-0.7804739 , -1.9946153 ,  0.07779949, ...,  0.8097438 ,
         0.16954414,  1.04959909],
       [-0.26446138, -1.04020355, -1.22396893, ..., -0.37033652,
        -1.70988399,  1.46570896],
       [-0.23229993,  2.01320972,  0.51248316, ...,  0.62603207,
        -0.74680501, -1.69474859]])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
sklearn_model_new = xgb.XGBClassifier(max_depth=5,
                                      learning_rate= 0.5, 
                                      verbosity=1, 
                                      objective='binary:logistic',
                                      random_state=1)

In [ ]:
sklearn_model_new.fit(X_train, y_train, 
                      early_stopping_rounds=10, 
                      eval_metric="error",
                      eval_set=[(X_test, y_test)])

### XGBoost网格搜索调参

In [ ]:
gsCv = GridSearchCV(sklearn_model_new,
                   {'max_depth': [4,5,6],
                    'n_estimators': [5,10,20]})
gsCv.fit(X_train,y_train)

In [ ]:
print(gsCv.best_score_)
print(gsCv.best_params_)

例子里的输出是：

0.9533333333333334

{'max_depth': 4, 'n_estimators': 10}

接着尝试在上面搜索的基础上调 learning_rate ：

In [ ]:
sklearn_model_new2 = xgb.XGBClassifier(max_depth=4,
                                       n_estimators=10,
                                       verbosity=1, 
                                       objective='binary:logistic',
                                       random_state=1)
gsCv2 = GridSearchCV(sklearn_model_new2, 
                     {'learning_rate ': [0.3,0.5,0.7]})
gsCv2.fit(X_train,y_train)

In [ ]:
print(gsCv2.best_score_)
print(gsCv2.best_params_)

例子里的输出是：

0.9516

{'learning_rate ': 0.3}

当然实际情况这里需要继续调参，这里假设我们已经调参完毕，我们尝试用验证集看看效果：

In [ ]:
sklearn_model_new2 = xgb.XGBClassifier(max_depth=4,
                                       learning_rate= 0.3, 
                                       verbosity=1, 
                                       objective='binary:logistic',
                                       n_estimators=10)

sklearn_model_new2.fit(X_train, y_train, 
                       early_stopping_rounds=10, 
                       eval_metric="error",
                       eval_set=[(X_test, y_test)])

最后的输出是：

validation_0-error : 0.0588

也就是验证集的准确率是 94.12%。

我们可以通过验证集的准确率来判断我们前面网格搜索调参是否起到了效果。实际处理的时候需要反复搜索参数并验证。